In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train=pd.read_csv('F:\\study\\ml\\DataSet\\Tap4fun\\tap_fun_train.csv',nrows=5)

In [3]:
df_train.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Columns: 109 entries, user_id to prediction_pay_price
dtypes: float64(13), int64(95), object(1)
memory usage: 4.7 KB


In [4]:
c_int=df_train.select_dtypes(include=['int64']).apply(pd.to_numeric,downcast='unsigned')
c_float=df_train.select_dtypes(include=['float']).apply(pd.to_numeric,downcast='float')

In [5]:
o_df=df_train.copy()

In [6]:
o_df[c_int.columns]=c_int
o_df[c_float.columns]=c_float

In [7]:
dtypes=o_df.dtypes
dtypes_col=dtypes.index
dtypes_type=[i.name for i in dtypes.values]
column_type=dict(zip(dtypes_col,dtypes_type))

In [8]:
#column_type

In [9]:
column_type.pop('register_time')

'object'

In [10]:
data_train=pd.read_csv('F:\\study\\ml\\DataSet\\Tap4fun\\tap_fun_train.csv',
                      dtype=column_type,parse_dates=['register_time'],infer_datetime_format=True)

In [ ]:
data_train.info(memory_usage='deep')

In [12]:
data_test=pd.read_csv('F:\\study\\ml\\DataSet\\Tap4fun\\tap_fun_test.csv',
                      dtype=column_type,parse_dates=['register_time'],infer_datetime_format=True)

In [15]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 828934 entries, 0 to 828933
Columns: 108 entries, user_id to pay_count
dtypes: datetime64[ns](1), float32(12), uint16(1), uint8(94)
memory usage: 120.2 MB


In [ ]:
data_test.to_csv

In [ ]:
data_test.info(memory_usage='deep')

In [ ]:
train_feat=data_train
test_feat=data_test


In [ ]:
train_feat.shape

In [ ]:
label_train=train_feat['prediction_pay_price']

In [ ]:
train_feat=train_feat.drop(['prediction_pay_price','register_time'],axis=1)

In [ ]:
train_feat.shape

In [ ]:
test_feat.shape

In [ ]:
#train_coorr1=train_feat.corr()

In [ ]:
#train_coorr1['prediction_pay_price'].sort_values(ascending=False).head(40)

In [ ]:
params={
    'boosting_type':'gbdt',
    'objective':'mse',
    'metric':'rmse',
    'min_child_weigth':3,
    'num_leaves':2**5,
    'lambda_l2':10,
    'subsample':0.7,
    'colsample_bytree':0.7,
    'colsample_bylevel':0.7,
    'learning_rate':0.05,
    'tree_method':'exact',
    'seed':2017,
    'nthread':4,
    'silent':True
}

In [ ]:
import time
import lightgbm as lgb

In [ ]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(train_feat,label_train,test_size=0.25,random_state=13)

In [ ]:
train=lgb.Dataset(X_train,y_train)
test=lgb.Dataset(X_test,label=y_test,reference=train)

In [ ]:
n=3000
gbm=lgb.train(params=params,
             train_set=train,
             num_boost_round=n,
             verbose_eval=50,
             valid_sets=[train,test])

In [ ]:
import gc
del train_feat
gc.collect()

In [ ]:
pred_train=gbm.predict(X_test)

In [ ]:
np.sqrt(mean_squared_error(y_test,pred_train))

In [ ]:
train_feat2=train_feat

In [ ]:
text_add=[]
for col in train_feat2.columns.values:
     if 'add' in col:
            text_add.append(col)

In [ ]:
text_add

In [ ]:
train_feat2['4add_value']=train_feat2['wood_add_value']+train_feat2['stone_add_value']+ \
train_feat2['ivory_add_value']+train_feat2['meat_add_value']

In [ ]:
train_feat2['prediction_pay_price']=data_train.prediction_pay_price

In [ ]:
text_reduce=[]
for col in train_feat2.columns.values:
     if 'reduce' in col:
            text_reduce.append(col)

In [ ]:
text_reduce

In [ ]:
train_feat2['4add_value']=train_feat2['wood_add_value']+train_feat2['stone_add_value']+ \
train_feat2['ivory_add_value']+train_feat2['meat_add_value']

In [ ]:
train_feat2['4reduce_value']=train_feat2['wood_reduce_value']+train_feat2['stone_reduce_value']+ \
train_feat2['ivory_reduce_value']+train_feat2['meat_reduce_value']

In [ ]:
train_corr2=train_feat2.corr()

In [ ]:
train_corr2.loc['4add_value','prediction_pay_price']

In [ ]:
train_corr2.loc['4reduce_value','prediction_pay_price']

In [ ]:
train_corr2['prediction_pay_price'].sort_values(ascending=False).head(40)